<a href="https://colab.research.google.com/github/rashida048/Tensorflow/blob/main/Regression_in_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/keras-regression-cnns/keras-regression-cnns.zip
!unzip -qq keras-regression-cnns.zip
%cd keras-regression-cnns

--2023-05-03 21:10:38--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/keras-regression-cnns/keras-regression-cnns.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.218.240.193, 52.92.164.194, 52.218.182.226, ...
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.218.240.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4583 (4.5K) [binary/octet-stream]
Saving to: ‘keras-regression-cnns.zip’

keras-regression-cn 100%[===================>]   4.48K  --.-KB/s    in 0s      

2023-05-03 21:10:39 (42.3 MB/s) - ‘keras-regression-cnns.zip’ saved [4583/4583]

/content/keras-regression-cnns


In [2]:
!git clone https://github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 39.06 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Updating files: 100% (2144/2144), done.


In [3]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import argparse
import locale
import glob
import cv2
import os

In [4]:
cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
inputPath = os.path.sep.join(["Houses-dataset/Houses Dataset/", "HousesInfo.txt"])
df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)

In [5]:
zipcodes = df["zipcode"].value_counts().keys().tolist()
counts = df['zipcode'].value_counts().tolist() 

for (zipcode, count) in zip(zipcodes, counts):
  if count < 25:
    idxs = df[df['zipcode'] == zipcode].index 
    df.drop(idxs, inplace=True)

In [6]:
df

,bedrooms,bathrooms,area,zipcode,price
30,5,3.0,2520,93446,789000
32,3,2.0,1802,93446,365000
39,3,3.0,2146,93446,455000
80,4,2.5,2464,91901,599000
81,2,2.0,1845,91901,529800
...,...,...,...,...,...
499,4,4.0,3000,93446,1495000
500,3,2.0,2330,93446,599900
501,3,2.5,1339,93446,344900
502,3,2.0,1472,93446,309995


In [9]:
images = []

for i in df.index.values:
  basepath = os.path.sep.join(["Houses-dataset/Houses Dataset/", "{}_*".format(i+1)])
  housepaths = sorted(list(glob.glob(basepath)))
  inputImages = []
  outputImage = np.zeros((64, 64, 3), dtype="uint8")

  for housePath in housepaths:
    image = cv2.imread(housePath)
    image = cv2.resize(image, (32, 32))
    inputImages.append(image)

  outputImage[0:32, 0:32] = inputImages[0]
  outputImage[0:32, 32:64] = inputImages[1]
  outputImage[32:64, 32:64] = inputImages[2]
  outputImage[32:64, 0:32] = inputImages[3]

  images.append(outputImage)

images= np.array(images) 

In [10]:
def load_house_images(df, inputPath):
	# initialize our images array (i.e., the house images themselves)
	images = []

	# loop over the indexes of the houses
	for i in df.index.values:
		# find the four images for the house and sort the file paths,
		# ensuring the four are always in the *same order*
		basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
		housePaths = sorted(list(glob.glob(basePath)))

		# initialize our list of input images along with the output image
		# after *combining* the four input images
		inputImages = []
		outputImage = np.zeros((64, 64, 3), dtype="uint8")

		# loop over the input house paths
		for housePath in housePaths:
			# load the input image, resize it to be 32 32, and then
			# update the list of input images
			image = cv2.imread(housePath)
			image = cv2.resize(image, (32, 32))
			inputImages.append(image)

		# tile the four input images in the output image such the first
		# image goes in the top-right corner, the second image in the
		# top-left corner, the third image in the bottom-right corner,
		# and the final image in the bottom-left corner
		outputImage[0:32, 0:32] = inputImages[0]
		outputImage[0:32, 32:64] = inputImages[1]
		outputImage[32:64, 32:64] = inputImages[2]
		outputImage[32:64, 0:32] = inputImages[3]

		# add the tiled image to our set of images the network will be
		# trained on
		images.append(outputImage)

	# return our set of images
	return np.array(images)

In [11]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1

	# define the model input
	inputs = Input(shape=inputShape)

	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input
		# appropriately
		if i == 0:
			x = inputs

		# CONV => RELU => BN => POOL
		x = Conv2D(f, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)

	# flatten the volume, then FC => RELU => BN => DROPOUT
	x = Flatten()(x)
	x = Dense(16)(x)
	x = Activation("relu")(x)
	x = BatchNormalization(axis=chanDim)(x)
	x = Dropout(0.5)(x)

	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)

	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)

	# construct the CNN
	model = Model(inputs, x)

	# return the CNN
	return model

In [12]:
args = {
	"dataset": "Houses-dataset/Houses Dataset/"
}

In [13]:
inputPath = os.path.sep.join([args["dataset"], "HousesInfo.txt"])
images = load_house_images(df, args["dataset"])
images = images / 255.0

In [14]:
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [15]:
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

In [18]:
import tensorflow as tf 
model = create_cnn(64, 64, 3, regress=True)
opt = tf.keras.optimizers.legacy.Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [19]:
model.fit(x=trainImagesX, y=trainY, 
    validation_data=(testImagesX, testY),
    epochs=200, batch_size=8)

Epoch 1/200
34/34 [==============================] - 4s 60ms/step - loss: 1561.3270 - val_loss: 101.7232
Epoch 2/200
34/34 [==============================] - 2s 55ms/step - loss: 1084.0439 - val_loss: 768.9060
Epoch 3/200
34/34 [==============================] - 2s 54ms/step - loss: 733.5787 - val_loss: 1392.7516
Epoch 4/200
34/34 [==============================] - 3s 80ms/step - loss: 602.6851 - val_loss: 2509.5779
Epoch 5/200
34/34 [==============================] - 2s 69ms/step - loss: 468.6579 - val_loss: 2802.7200
Epoch 6/200
34/34 [==============================] - 2s 54ms/step - loss: 404.6313 - val_loss: 2422.0564
Epoch 7/200
34/34 [==============================] - 2s 52ms/step - loss: 343.3018 - val_loss: 2503.7510
Epoch 8/200
34/34 [==============================] - 2s 55ms/step - loss: 327.9789 - val_loss: 3784.7974
Epoch 9/200
34/34 [==============================] - 2s 54ms/step - loss: 302.8687 - val_loss: 2801.0820
Epoch 10/200
34/34 [==============================] - 2

In [20]:
preds = model.predict(testImagesX)

# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

3/3 [==============================] - 0s 42ms/step
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 61.26%, std: 51.36%
